<a href="https://jupyterhub.user.eopf.eodc.eu/hub/login?next=%2Fhub%2Fspawn%3Fnext%3D%252Fhub%252Fuser-redirect%252Fgit-pull%253Frepo%253Dhttps%253A%252F%252Fgithub.com%252Feopf-toolkit%252Feopf-101%2526branch%253Dmain%2526urlpath%253Dlab%252Ftree%252Feopf-101%252F04_eopf_and_stac%252F43_eopf_stac_connection.ipynb%23fancy-forms-config=%7B%22profile%22%3A%22choose-your-environment%22%2C%22image%22%3A%22unlisted_choice%22%2C%22image%3Aunlisted_choice%22%3A%224zm3809f.c1.de1.container-registry.ovh.net%2Feopf-toolkit-python%2Feopf-toolkit-python%3Alatest%22%2C%22autoStart%22%3A%22true%22%7D" target="_blank">
  <button style="background-color:#0072ce; color:white; padding:0.6em 1.2em; font-size:1rem; border:none; border-radius:6px; margin-top:1em;">
    🚀 Launch this notebook in JupyterLab
  </button>
</a>

### Introduction

In this section, we will dive into the programmatic access of EOPF Zarr Collections available in the [EOPF Sentinel Zarr Sample Service STAC Catalog](https://stac.browser.user.eopf.eodc.eu/?.language=en).
We will introduce Python libraries that enable us to effectively access and search through STAC catalogs.

### What we will learn
- 🔍 How to **programmatically browse** through available collections available via the EOPF Zarr STAC Catalog
- 📊 Understanding **collection metadata** in user-friendly terms
- 🎯 **Searching for specific data** with help of the `pystac` and `pystac-client` libraries.

### Prerequisites

For this tutorial, we will make use of the [pystac](https://pystac.readthedocs.io/en/stable/) and [pystac_client](https://pystac-client.readthedocs.io/en/latest/api.html) Python libraries that facilitate the programmatic access and efficient search of a STAC Catalog.

<hr>

#### Import libraries

In [ ]:
import requests
from typing import List, Optional, cast
from pystac import Collection, MediaType
from pystac_client import Client, CollectionClient
from datetime import datetime

#### Helper functions

##### `list_found_elements`
As we are expecting to visualise several elements that will be stored in lists, we define a function that will allow us retrieve item `id`'s and collections `id`'s for further retrieval.

In [ ]:
from pystac import Item

def list_found_elements(search_result):
    """
    Retrieve item IDs and collection IDs from search results.
    Uses pages_as_dicts() to handle pagination and cleans items 
    with missing href in assets (workaround for STAC API issues).
    """
    id_list = []
    coll_list = []
    
    for page_dict in search_result.pages_as_dicts():
        for feature in page_dict.get("features", []):
            # Clean assets with missing href before parsing
            if "assets" in feature:
                feature["assets"] = {
                    key: asset for key, asset in feature["assets"].items() if "href" in asset
                }
            
            # Now parse the cleaned item
            try:
                item = Item.from_dict(feature)
                id_list.append(item.id)
                coll_list.append(item.collection_id)
            except Exception as e:
                item_id = feature.get("id", "unknown")
                print(f"⚠️  Skipping item {item_id}: {e}")
                continue
    
    return id_list, coll_list

<hr>

## Establish a connection to the EOPF Zarr STAC Catalog
Our first step is to establish a connection to the EOPF Sentinel Zarr Sample Service STAC Catalog. For this, you need the Catalog's base URL, which you can find on the web interface under the **API & URL** tab. By clicking on 🔗**Source**, you will get the address of the STAC metadata file - which is available [here](https://stac.core.eopf.eodc.eu/).

![EOPF API url for connection](./../img/api_connection.png)

Copy paste the URL: `https://stac.core.eopf.eodc.eu/`.

With the `Client.open()` function, we can create the access to the starting point of the Catalog by providing the specific url. 
If the connection was successful, you will see the description of the STAC catalog and additional information.

In [ ]:
eopf_stac_api_root_endpoint = "https://stac.core.eopf.eodc.eu/" #root starting point
eopf_catalog = Client.open(url=eopf_stac_api_root_endpoint) # calls the selected url
eopf_catalog

Congratulations. We successfully connected to the EOPF Zarr STAC Catalog, and we can now start exploring its content.

## Explore available collections

Once a connection established, the next logical step is to get an overview of all the collections the STAC catalog offers. We can do this with the function `get_all_collections()`. The result is a list, which we can loop through to print the relevant collection IDs.

**Please note:** Since the EOPF Zarr STAC Catalog is still in active development, we need to test whether a collection is valid, otherwise you might get an error message. The code below is testing for validity and for one collection, it throws an error. 

You see, that so far, we can browse through 10 available collections

In [ ]:
try:
    for collection in eopf_catalog.get_all_collections():
        print(collection.id)

except Exception:
    print(
        "* [https://github.com/EOPF-Sample-Service/eopf-stac/issues/18 appears to not be resolved]"
    )

In a next step, we can select one `collection` and retrieve certain metadata that allow us to get more information about the selected collection, such as keywords, the ID and useful links for resources.

In [ ]:
S2l2a_coll = eopf_catalog.get_collection('sentinel-2-l2a')
print('Keywords:        ',S2l2a_coll.keywords)
print('Catalog ID:      ',S2l2a_coll.id)
print('Available Links: ',S2l2a_coll.links)

## Searching inside the EOPF STAC API

With the `.search()` function of the `pystac-client` library, we can search inside a STAC catalog we established a connection with. We can filter based on a series of parameters to tailor the search for available data for a specific time period and geographic bounding box. 


### Filter for temporal extent

Let us search on the `datetime` parameter. For this, we specify the `datetime` argument for a time period we are interested in, e.g. from 1 May 2020 to 31 May 2023. In addition, we also specify the `collection` parameter indicating that we only want to search for the Sentinel-2 L2A collection. 

We apply the helper function `list_found_elements` which constructs a list from the search result. If we check the length of the final list, we can see that for the specified time period, 196 items were found.

In [ ]:
time_frame = eopf_catalog.search(  #searching the catalog
    collections='sentinel-2-l2a',
    datetime="2020-05-01T00:00:00Z/2023-05-31T23:59:59.999999Z")  # the interval we are interested in, separated by '/'

# we apply the helper function `list_found_elements`
time_items=list_found_elements(time_frame)
print(time_frame)

print("Search Results:")
print('Total Items Found for Sentinel-2 L-2A between May 1, 2020, and May 31, 2023:  ',len(time_items[0]))


### Filter for spatial extent
Now, let us filter based on a specific area of interest. We can use the `bbox` argument, which is composed by providing the top-left and bottom-right corner coordinates. It is similar to drawing the extent in the interactive map of the EOPF browser interface.

For example, we defined a bounding box of the outskirts of Innsbruck, Austria. We then again apply the helper function `list_found_elements` and see that for the defined area, only 39 items are available.

In [ ]:
bbox_search =  eopf_catalog.search(  #searching the catalog
    collections='sentinel-2-l2a',
    bbox=(
        11.124756, 47.311058, #top left
        11.459839, 47.463624  #bottom-right
        )
)

innsbruck_sets=list_found_elements(bbox_search) #we apply our constructed function that stores internal information

#Results
print("Search Result:")
print('Total Items Found:  ',len(innsbruck_sets[0]))

### Combined filtering: Collection + temporal extent + spatial extent
As a usual workflow, we often look for datasets within an AOI and a specific period of time. The `search()` function allows us also to combine the `collection`, `bbox` and `datetime` arguments in one search request.

Let us now search for Items available for the AOI around Innsbruck within the previously defined timeframe for the **Sentinel-2 Level-2A** collection. As a result, we get 27 Items that are available for our selection.

In [ ]:
innsbruck_s2 = eopf_catalog.search( 
    collections= 'sentinel-2-l2a', # interest Collection,
    bbox=(11.124756, 47.311058, # AOI extent
          11.459839,47.463624),
    datetime='2020-05-01T00:00:00Z/2025-05-31T23:59:59.999999Z' # interest period
)

combined_ins =list_found_elements(innsbruck_s2)

print("Search Results:")
print('Total Items Found for Sentinel-2 L-2A over Innsbruck:  ',len(combined_ins[0]))

Let's repeat a combined search for a different collection.<br>
We define a new AOI for the coastal area of Rostock, Germany and we search over the **Sentinel-3 SLSTR-L2** collection for the same time period as above.<br>

As a result, 14 Items are available for the specified search.

In [ ]:
rostock_s3 = eopf_catalog.search(
    bbox=(11.766357,53.994566, # AOI extent
          12.332153,54.265086),
    collections= ['sentinel-3-slstr-l2-lst'], # interest Collection
    datetime='2020-05-01T00:00:00Z/2025-05-31T23:59:59.999999Z' # interest period
)

combined_ros=list_found_elements(rostock_s3)

print("Search Results:")
print('Total Items Found for Sentinel-3 SLSTR-L2 over Rostock Coast:  ',len(combined_ros[0]))

### Retrieve Asset URLs for accessing the data

So far, we have made a search among the STAC catalog and browsed over the general metadata of the collections. To access the actual EOPF Zarr `Items`, we need to get their storage location in the cloud.

The relevant information we can find inside the `.items` argument by the `.get_assets()` function. Inside, it allows us to specify the `.MediaType` we are interested in. In our example, we want to obtain the location of the `.zarr` file.

Let us retrieve the `url` of the 27 available items over Innsbruck. The resulting URL we can then use to directly access an asset in our workflow.

In [ ]:
def get_item_cleaned(collection, item_id):
    """
    Retrieve an item from a collection and clean assets with missing href.
    Workaround for STAC API issues where some assets lack href attribute.
    """
    import requests
    
    # Build the item URL and fetch raw JSON
    items_href = collection.get_single_link("items").href
    item_url = f"{items_href.rstrip('/')}/{item_id}"
    
    response = requests.get(item_url)
    response.raise_for_status()
    item_dict = response.json()
    
    # Clean assets with missing href
    if "assets" in item_dict:
        item_dict["assets"] = {
            key: asset for key, asset in item_dict["assets"].items() if "href" in asset
        }
    
    return Item.from_dict(item_dict)

# Retrieve assets for Innsbruck items
assets_loc = []
for item_id in combined_ins[0]:
    item = get_item_cleaned(S2l2a_coll, item_id)
    zarr_assets = item.get_assets(media_type=MediaType.ZARR)
    assets_loc.append(zarr_assets)

first_item = assets_loc[0]  # we select the first item from our list

print("Search Results:")
print('URL for accessing', combined_ins[0][0], 'item:  ', first_item['product'])

### Retrieve Item metadata

Finally, once you selected an `Item`, you can also explore the relevant metadata on `Item` level. For example with the `keys()` function, you can retrieve the available assets of the selected Item.

In [ ]:
print('Available Assets: ', list(first_item.keys()))

## 💪 Now it is your turn

The following expercises will help you master the STAC API and understand how to find the data you need.


### Task 1: Explore Your Area of Interest

* Go to [http://bboxfinder.com/](http://bboxfinder.com/) and select an area of interest (AOI) (e.g. your hometown, a research site, etc.)
* Copy the bounding box coordinates of your area of interest
* Change the provided code above to search for data over your AOI

### Task 2: Temporal Analysis

* Compare data availability across different years for the **Sentinel-2 L-2A Collection**.
* Search for items in the year 2022
* Repeat the search for the year 2024

### Task 3: Explore the SAR Mission and combine multiple criteria

* Do the same for a different `Collection`, the **Sentinel-1 Level-1 GRD**, e.g. you can use the ID `sentinel-1-l1-grd`
* How many assets are available for the year 2024?

## Conclusion

This tutorial has provided a clear and practical introduction on how you can programmatically access and search through [EOPF Sentinel Zarr Sample Service STAC API](https://stac.browser.user.eopf.eodc.eu/?.language=en).


## What's next?

In the following [notebook](./44_eopf_stac_xarray_tutorial.ipynb), we will explore how to retrieve an Item of our interest, based on several parameters and load the actual data array as `xarray`.
